# Run Full TSB-CL on Kaggle (Using GitHub Code)

This notebook is designed to run on Kaggle. It performs the following steps:
1.  **Clones the GitHub repository** containing the project code.
2.  **Compiles the MSBE** (Maximal Similar Biclique Enumeration) C++ tool.
3.  **Runs the training script** by importing the code directly from the cloned repository.

This ensures you are always running the latest version of the code without manually copying large blocks of text.

In [ ]:
# 1. Clone Repository
# Remove existing folder to ensure fresh clone
!rm -rf ai_project 
!git clone https://github.com/yangzeha/ai_project.git

# 2. Install Missing Dependencies
# Fix for potential NVIDIA repo GPG error on Kaggle
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/3bf863cc.pub
!apt-get update || echo "Apt-get update had some errors, proceeding to install..."
!apt-get install -y libsparsehash-dev

# 3. Compile MSBE (C++ Tool)
import os
import subprocess
import sys

# Define paths
PROJECT_DIR = os.path.abspath("ai_project")
MSBE_DIR = os.path.join(PROJECT_DIR, "Similar-Biclique-Idx-main")
MSBE_EXE_PATH = os.path.join(MSBE_DIR, "msbe")

# Check if directory exists
if os.path.exists(MSBE_DIR):
    print(f"Compiling MSBE in {MSBE_DIR}...")
    
    # Clean up
    if os.path.exists(MSBE_EXE_PATH):
        if os.path.isdir(MSBE_EXE_PATH):
            subprocess.run(["rm", "-rf", "msbe"], cwd=MSBE_DIR)
        else:
            os.remove(MSBE_EXE_PATH)

    # Compile
    cmd = ["g++", "-O3", "main.cpp", "-o", "msbe", "-D_PrintResults_"]
    try:
        subprocess.run(cmd, cwd=MSBE_DIR, check=True)
        subprocess.run(["chmod", "+x", "msbe"], cwd=MSBE_DIR, check=True)
        print("✅ Compilation successful!")
    except subprocess.CalledProcessError as e:
        print(f"❌ Compilation failed: {e}")
        # Stop execution if compilation fails
        sys.exit(1)
else:
    print(f"❌ Error: Directory {MSBE_DIR} not found.")
# 4. Run Training
# This script trains the Full TSB-CL model (with Residual Connection fix)
# It will run for 50 epochs and save the best model to 'full_tsbcl_best.pth'

!python ai_project/TSB_CL_Project/train_full_kaggle.py

In [ ]:
# 5. Run Model Comparison (Baseline vs TSB-CL)
# This script runs both the baseline (Pure LightGCN) and the enhanced model (Full TSB-CL) 
# to compare their performance on the same data split.

# Ensure we have the latest code
!cd ai_project && git pull

# Run comparison script
!python ai_project/TSB_CL_Project/compare_models_kaggle.py